# 오차역전파법 backpropagation
- 계산그래프
- 연쇄법칙
- 역전파
- 단순한 계층 구현
- 활성화 함수 계층 구현
- Affine/Softmax 계층 구현
- 오차역전파법 구현하기

## 개요 

4장에서는 신경망 매개변수의 기울기(정확히는 W, b들에 대한 손실 함수의 기울기)를 수치 미분을 통해 구했다.
단순하고 구현이 쉽지만 계산 시간이 오래 걸린다.
> 진짜 1만번 반복중 1번 반복에 30초씩 걸렸는데 오차역전파법을 사용하니 5초만에 1만번 반복이 끝났다.


## 계산그래프

## 연쇄법칙

## 역전파

## 단순한 계층 구현

## 활성화 함수 계층 구현

## Affine/Softmax 계층 구현

## 오차역전파법 구현하기